In [1]:
import random
random.seed(10)

import os
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Activation, Dropout, BatchNormalization
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
#from keras.applications import InceptionV3, ResNet50, InceptionResNetV2, Xception
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from keras.callbacks import  Callback, EarlyStopping

!pip install wandb
import wandb
from wandb.keras import WandbCallback
!pip install wandb --upgrade

In [2]:
# Prepare the dataset for training and testing
def prepare_dataset(DATA_DIR="inaturalist_12K", augment_data=False, batch_size=100):
    train_dir = os.path.join(DATA_DIR, "train")
    test_dir = os.path.join(DATA_DIR, "val")

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [3]:
input_shape = (200,200,3)
image_size = input_shape[0]

def train():
    
    config_defaults = {
        'freeze_before' : 70,
        'models' : 'IRNV2',
        'dropout': 0.1,
        'batch_size': 64,
        'dense_size': 64,
        'num_classes': 10,
        'epochs' : 10,
        'augment_data': True,
        'lr': 0.0001
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    config = wandb.config
    
    
    # Defining models and run-name
    if config.models == 'IV3':
        base_model =  keras.applications.InceptionV3(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_IncV3__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'IRNV2':
        base_model =  keras.applications.InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_IncResNetV2__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'RN50':
        base_model =  keras.applications.ResNet50(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_ResNet50__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'XCP':
        base_model =  keras.applications.Xception(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_Xception__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)

        
    base_model.trainable = False

    stack_model = Sequential()
    stack_model.add(base_model)
    stack_model.add(Flatten())
    stack_model.add(Dense(config.dense_size, activation='relu'))
    stack_model.add(Dropout(config.dropout))
    stack_model.add(Dense(config.num_classes, activation='softmax'))
    
    final_model = stack_model
    final_model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="accuracy")
    
    train_generator, val_generator, test_generator = prepare_dataset(augment_data=config.augment_data, batch_size=config.batch_size)
    
    final_model.fit(train_generator,
                    batch_size = config.batch_size,
                    epochs=config.epochs,
                    validation_data=val_generator,
                    callbacks=[WandbCallback(),keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

    base_model.trainable = True

    # Freeze layers
    freeze_point = len(base_model.layers) - config.freeze_before
    for layer in base_model.layers[:freeze_point]:
        layer.trainable = False
    
    final_model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="accuracy")

    final_model.fit(train_generator,
                    batch_size = config.batch_size,
                    epochs = 7,
                    validation_data=val_generator,
                    callbacks = [WandbCallback(),keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
    
    #Testing model
    
    loss, acc = final_model.evaluate(test_generator, batch_size=config.batch_size)

    final_model.save("best_model_p2.h5")

In [4]:
sweep_config = {
    'method': 'grid', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        
        'freeze_before' : {
            'values': [70]
        },
        'models' :{
            'values' : ['IRNV2']
        },
        'dropout': {
            'values': [0.1]
        },     
        'batch_size': {
            'values': [64]
        },
        'dense_size':{
            'values': [64]
        },
        'augment_data': {
            "values": [True]
        },
        'epochs': {
            "values": [10]
        },
        'lr': {
            "values": [0.0001]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="DL_CS6910_Assignment2")

Create sweep with ID: tmho2xlz
Sweep URL: https://wandb.ai/cs21m029_keyur_raval/DL_CS6910_Assignment2/sweeps/tmho2xlz


In [5]:
wandb.agent(sweep_id, train, count = 1)

wandb: Agent Starting Run: n6ove3ou with config:
wandb: 	augment_data: True
wandb: 	batch_size: 64
wandb: 	dense_size: 64
wandb: 	dropout: 0.1
wandb: 	epochs: 10
wandb: 	freeze_before: 70
wandb: 	lr: 0.0001
wandb: 	models: IRNV2
wandb: Currently logged in as: cs21m029_keyur_raval (use `wandb login --relogin` to force relogin)


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/10
141/141 [==============================] - 187s 1s/step - loss: 1.4642 - accuracy: 0.5358 - val_loss: 1.1567 - val_accuracy: 0.6356
Epoch 2/10
141/141 [==============================] - 157s 1s/step - loss: 1.1612 - accuracy: 0.6273 - val_loss: 1.0670 - val_accuracy: 0.6456
Epoch 3/10
141/141 [==============================] - 156s 1s/step - loss: 1.0903 - accuracy: 0.6413 - val_loss: 0.9721 - val_accuracy: 0.7007
Epoch 4/10
141/141 [==============================] - 155s 1s/step - loss: 1.0530 - accuracy: 0.6559 - val_loss: 1.0362 - val_accuracy: 0.6657
Epoch 5/10
141/141 [==============================] - 155s 1s/step - loss: 1.0147 - accuracy: 0.6673 - val_loss: 1.0259 - val_accuracy: 0.6617
Epoch 6/10
141/141 [==============================] - 155s 1s/step - loss: 0.9866 - accuracy: 0.6758 - val_loss: 1.0247 - val_accuracy: 0.6476
Epoch 7/10
141

accuracy,▁▃▄▄▄▄▅▅▅▅▅▆▆▇▇██
epoch,▁▂▃▃▄▅▆▆▇█▁▂▃▃▄▅▆
loss,█▆▅▅▅▄▄▄▄▄▄▃▃▂▂▁▁
val_accuracy,▁▂▆▃▃▂▂▄▄▄▆▅▇▇▇▇█
val_loss,█▆▄▅▅▅▅▅▄▅▃▃▂▂▃▃▁
accuracy,0.81967
best_epoch,6
best_val_loss,0.82279
epoch,6
loss,0.51868
val_accuracy,0.73273
